In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark import __version__


# import matplotlib.pyplot as plt
# import pyspark.pandas as ps
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

print("PySpark", __version__)


PySpark 3.2.1


In [2]:

# spark = SparkSession.builder \
    # .master('local[*]') \
#     .appName("Spark ML") \
#     .master("spark://10.32.45.215:7077") \
#     .config("spark.executor.memory", "24g") \
#     .getOrCreate()
  # .config("spark.dynamicAllocation.enabled", "true") \
  # .config("spark.shuffle.service.enabled", "true") \

spark = SparkSession.builder \
  .master("spark://carloshkayser:7077") \
  .config("spark.driver.memory", "6g") \
  .config("spark.executor.instances", "4") \
  .config("spark.executor.memory", "4g") \
  .config("spark.executor.cores", "2") \
  .getOrCreate()

# .config("spark.driver.memory", "24g") \
# .config("spark.kryoserializer.buffer.max", "512m") \

spark

22/06/06 03:13:21 WARN Utils: Your hostname, carloshkayser resolves to a loopback address: 127.0.1.1; using 10.32.45.215 instead (on interface ens160)
22/06/06 03:13:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/06 03:13:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/06 03:13:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# from pyspark import SparkContext, SparkConf
# from pyspark.sql import SparkSession

# sparkConf = SparkConf()

# # sparkConf.setMaster("k8s://kubernetes.default.svc.cluster.local")
# sparkConf.setMaster("k8s://https://192.168.49.2:8443") # minikube ip

# sparkConf.setAppName("KUBERNETES-IS-AWESOME")
# sparkConf.set("spark.kubernetes.container.image", "kayser/spark-py:3.2.1-hadoop-3.2.0")
# sparkConf.set("spark.kubernetes.namespace", "spark")

# sparkConf.set("spark.executor.instances", "2")
# sparkConf.set("spark.executor.memory", "4g")
# sparkConf.set("spark.executor.cores", "2")
# sparkConf.set("spark.driver.blockManager.port", "7777")
# sparkConf.set("spark.driver.port", "2222")
# # sparkConf.set("spark.driver.host", "jupyter.spark.svc.cluster.local")
# # sparkConf.set("spark.driver.bindAddress", "0.0.0.0")

# spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
# spark

In [4]:
from pyspark.sql.types import StructType, StructField, FloatType, StringType, LongType, IntegerType, DoubleType

schema = StructType([
    StructField('id', LongType(), True),
    StructField("click", FloatType(), True),
    StructField("hour", IntegerType(), True),
    StructField("C1", IntegerType(), True),
    StructField("banner_pos", IntegerType(), True),
    StructField("site_id", StringType(), True),
    StructField("site_domain", StringType(), True),
    StructField("site_category", StringType(), True),
    StructField("app_id", StringType(), True),
    StructField("app_domain", StringType(), True),
    StructField("app_category", StringType(), True),
    StructField("device_id", StringType(), True),
    StructField("device_ip", StringType(), True),
    StructField("device_model", StringType(), True),
    StructField("device_type", IntegerType(), True),
    StructField("device_conn_type", IntegerType(), True),
    StructField("C14", IntegerType(), True),
    StructField("C15", IntegerType(), True),
    StructField("C16", IntegerType(), True),
    StructField("C17", IntegerType(), True),
    StructField("C18", IntegerType(), True),
    StructField("C19", IntegerType(), True),
    StructField("C20", IntegerType(), True),
    StructField("C21", IntegerType(), True)
])

In [5]:
# Load training data
# training = spark.read.format("csv") \
#     .option("header", "true") \
#     .option("inferSchema", "true") \
#     .load('../dataset/click-through-rate-prediction/train.gz')

df = spark.read.format("csv") \
    .option("header", "true") \
    .schema(schema) \
    .load('../dataset/click-through-rate-prediction/train.gz')


In [6]:
df = df.limit(1000000)

In [7]:
df = df.na.drop() 

df = df.withColumnRenamed("click", "label")


In [8]:
df.count()

22/06/06 03:14:15 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/06/06 03:14:30 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/06/06 03:14:45 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/06/06 03:15:00 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/06/06 03:15:15 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


469294

In [9]:
test_data, raw_training_data = df.randomSplit([0.3, 0.7])


In [10]:
# raw_training_data = training.limit(1000)

# raw_training_data = raw_training_data.na.drop() 

In [11]:
# raw_training_data = raw_training_data.withColumnRenamed("click", "label")

raw_training_data.printSchema()


root
 |-- id: long (nullable = true)
 |-- label: float (nullable = true)
 |-- hour: integer (nullable = true)
 |-- C1: integer (nullable = true)
 |-- banner_pos: integer (nullable = true)
 |-- site_id: string (nullable = true)
 |-- site_domain: string (nullable = true)
 |-- site_category: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- app_domain: string (nullable = true)
 |-- app_category: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- device_ip: string (nullable = true)
 |-- device_model: string (nullable = true)
 |-- device_type: integer (nullable = true)
 |-- device_conn_type: integer (nullable = true)
 |-- C14: integer (nullable = true)
 |-- C15: integer (nullable = true)
 |-- C16: integer (nullable = true)
 |-- C17: integer (nullable = true)
 |-- C18: integer (nullable = true)
 |-- C19: integer (nullable = true)
 |-- C20: integer (nullable = true)
 |-- C21: integer (nullable = true)



In [12]:
# # raw_training_data = raw_training_data.limit(1000)

# raw_training_data = raw_training_data.withColumnRenamed("click", "label")

# raw_training_data = raw_training_data.na.drop()

In [13]:
from pyspark.ml.classification import LogisticRegression

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import MinMaxScaler

In [14]:
# Create the logistic regression model
lr = LogisticRegression(maxIter=10, regParam= 0.01)

In [15]:
# Create a StringIndexer
from pyspark.ml.feature import StringIndexer

cols = []
pipeline_stages = []
feature_columns = []

for name, type in raw_training_data.dtypes:
    if type == "string":
        feature_columns.append(f"{name}Index")
        pipeline_stages.append(StringIndexer(inputCol=name, outputCol=f"{name}Index", handleInvalid="skip"))
    
        cols.append(f"{name}Index")
    
    else:
        cols.append(name)


In [16]:
# Create a one hot encoder
feature_columns = ['site_idIndex', 'site_domainIndex', 'site_categoryIndex', 'app_idIndex', 'app_domainIndex', 'app_categoryIndex', 'device_idIndex', 'device_ipIndex', 'device_modelIndex']
output_ohe_columns = ['site_id_ohe', 'site_domain_ohe', 'site_category_ohe', 'app_id_ohe', 'app_domain_ohe', 'app_category_ohe', 'device_id_ohe', 'device_ip_ohe', 'device_model_ohe']

ohe = OneHotEncoder(inputCols = feature_columns, outputCols = output_ohe_columns)


In [17]:
from pyspark.ml.feature import MinMaxScaler

# Input list for scaling
inputs = ["hour", "C1", "banner_pos", "device_type", "device_conn_type", "C14", "C15", "C16", "C17", "C18", "C19", "C20", "C21"]

# We scale our inputs
assembler1 = VectorAssembler(inputCols=inputs, outputCol="features_scaled1")
scaler = MinMaxScaler(inputCol="features_scaled1", outputCol="features_scaled")


In [18]:
# We create a second assembler for the encoded columns.
assembler2 = VectorAssembler(
  inputCols=['features_scaled'] + output_ohe_columns, outputCol="features"
)


In [19]:
from pyspark.ml import Pipeline

# Create stages list
myStages = pipeline_stages + [assembler1, scaler, ohe, assembler2, lr]

# Set up the pipeline
pipeline = Pipeline(stages= myStages)

# We fit the model using the training data.
pModel = pipeline.fit(raw_training_data)

# We transform the data.
trainingPred = pModel.transform(raw_training_data)

# # We select the actual label, probability and predictions
trainingPred.select('label', 'probability', 'prediction').show()

22/06/06 03:17:07 WARN DAGScheduler: Broadcasting large task binary with size 1172.5 KiB
22/06/06 03:17:23 WARN DAGScheduler: Broadcasting large task binary with size 4.9 MiB
22/06/06 03:17:25 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/06/06 03:17:35 WARN DAGScheduler: Broadcasting large task binary with size 5.0 MiB
22/06/06 03:18:03 WARN DAGScheduler: Broadcasting large task binary with size 18.4 MiB
22/06/06 03:18:12 WARN DAGScheduler: Broadcasting large task binary with size 18.4 MiB
22/06/06 03:18:20 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/06/06 03:18:20 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/06/06 03:18:20 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/06/06 03:18:20 WARN BLAS: Fai

+-----+--------------------+----------+
|label|         probability|prediction|
+-----+--------------------+----------+
|  0.0|[0.99045397072254...|       0.0|
|  0.0|[0.80807590059215...|       0.0|
|  0.0|[0.98711788683391...|       0.0|
|  0.0|[0.70260273564967...|       0.0|
|  0.0|[0.87271516926943...|       0.0|
|  1.0|[0.01903280412963...|       1.0|
|  1.0|[0.05092559098070...|       1.0|
|  1.0|[0.47611269367332...|       1.0|
|  0.0|[0.98773308401171...|       0.0|
|  0.0|[0.98117584756017...|       0.0|
|  0.0|[0.98089943122165...|       0.0|
|  0.0|[0.99139140112180...|       0.0|
|  0.0|[0.97994919525048...|       0.0|
|  0.0|[0.82804539057072...|       0.0|
|  0.0|[0.48790869933666...|       1.0|
|  0.0|[0.99032445620101...|       0.0|
|  0.0|[0.99228023307818...|       0.0|
|  0.0|[0.93538059880082...|       0.0|
|  0.0|[0.98595734351784...|       0.0|
|  0.0|[0.93120113551140...|       0.0|
+-----+--------------------+----------+
only showing top 20 rows



In [20]:
pModel.save("model/spark-logistic-regression-model")

22/06/06 03:19:11 WARN TaskSetManager: Stage 94 contains a task of very large size (2411 KiB). The maximum recommended task size is 1000 KiB.
22/06/06 03:19:13 WARN TaskSetManager: Stage 112 contains a task of very large size (1565 KiB). The maximum recommended task size is 1000 KiB.


In [21]:
pred = pModel.transform(test_data)

In [22]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(pred)

print("Train Accuracy = %g " % (accuracy))


22/06/06 03:19:27 WARN DAGScheduler: Broadcasting large task binary with size 19.9 MiB


Train Accuracy = 0.83162 


In [ ]:
stop

In [24]:
pred.printSchema()

root
 |-- id: long (nullable = true)
 |-- label: float (nullable = true)
 |-- hour: integer (nullable = true)
 |-- C1: integer (nullable = true)
 |-- banner_pos: integer (nullable = true)
 |-- site_id: string (nullable = true)
 |-- site_domain: string (nullable = true)
 |-- site_category: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- app_domain: string (nullable = true)
 |-- app_category: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- device_ip: string (nullable = true)
 |-- device_model: string (nullable = true)
 |-- device_type: integer (nullable = true)
 |-- device_conn_type: integer (nullable = true)
 |-- C14: integer (nullable = true)
 |-- C15: integer (nullable = true)
 |-- C16: integer (nullable = true)
 |-- C17: integer (nullable = true)
 |-- C18: integer (nullable = true)
 |-- C19: integer (nullable = true)
 |-- C20: integer (nullable = true)
 |-- C21: integer (nullable = true)
 |-- site_idIndex: double (nullable = false)
 |-- site

In [36]:
from pyspark.sql.types import StructType, StructField, FloatType, StringType, LongType, IntegerType, DoubleType
from pyspark.sql.functions import split, regexp_replace

pred = pred.withColumn("probability", pred["probability"].cast("String")) \
    .withColumn('probabilityre', split(regexp_replace("probability", "^\[|\]", ""), ",")[1].cast(DoubleType()))

In [38]:
pred.select("id", "rawPrediction", "probability", "prediction", "probabilityre").limit(10).toPandas()

22/06/06 05:06:07 WARN DAGScheduler: Broadcasting large task binary with size 19.9 MiB


,id,rawPrediction,probability,prediction,probabilityre
0,9984920344968,"[1.0216997729415782, -1.0216997729415782]","[0.7353035623637518,0.26469643763624817]",0.0,0.264696
1,441039161952503,"[1.1711235751101627, -1.1711235751101627]","[0.7633480471593245,0.23665195284067553]",0.0,0.236652
2,488135999371288,"[4.130848399311453, -4.130848399311453]","[0.9841848967820319,0.015815103217968085]",0.0,0.015815
3,539983806946224,"[4.600912915693666, -4.600912915693666]","[0.9900571888527,0.009942811147300001]",0.0,0.009943
4,1013507549820690,"[2.1518648867926085, -2.1518648867926085]","[0.8958429149576012,0.10415708504239884]",0.0,0.104157
5,1027051155424902,"[3.027562002069021, -3.027562002069021]","[0.9538038685505382,0.046196131449461775]",0.0,0.046196
6,1200637710450539,"[0.09946731762671135, -0.09946731762671135]","[0.5248463474935845,0.4751536525064155]",0.0,0.475154
7,1233274924027154,"[6.547184902585121, -6.547184902585121]","[0.9985679066476261,0.001432093352373931]",0.0,0.001432
8,1272541125281255,"[3.9749550021280777, -3.9749550021280777]","[0.9815660458185808,0.018433954181419243]",0.0,0.018434
9,1471506830732833,"[3.9688689641686192, -3.9688689641686192]","[0.9814556008197808,0.018544399180219218]",0.0,0.018544


In [30]:
from pyspark.ml.feature import VectorSlicer
from pyspark.ml.linalg import Vectors
from pyspark.sql.types import Row

slicer = VectorSlicer(inputCol="probability", outputCol="probability_", indices=[1])

pred = slicer.transform(pred)


In [ ]:
raw_training_data.columns


In [ ]:
# training = training.select([col for col in training.columns if col != "id"])
raw_training_data.limit(10).toPandas()

In [ ]:
raw_training_data.describe().toPandas()

In [ ]:
# Get string columns
for name, type in raw_training_data.dtypes:
  print(name, type)

In [ ]:
# from pyspark.ml.feature import Imputer

# imputer = Imputer(
#   inputCols=["Glucose","BloodPressure","SkinThickness","BMI","Insulin"], 
#   outputCols=["Glucose","BloodPressure","SkinThickness","BMI","Insulin"]
# )

# model = imputer.fit(raw_training_data)
# raw_training_data = model.transform(raw_training_data)

# raw_training_data.show(5)

In [ ]:
from pyspark.ml.feature import StringIndexer

cols = []
pipeline_stages = []
feature_columns = []

for name, type in raw_training_data.dtypes:
    if type == "string":
        feature_columns.append(f"{name}Index")
        pipeline_stages.append(StringIndexer(inputCol=name, outputCol=f"{name}Index"))
    
        cols.append(f"{name}Index")
    
    else:
        cols.append(name)

print(feature_columns)

from pyspark.ml import Pipeline

raw_training_data = Pipeline(stages=pipeline_stages).fit(raw_training_data).transform(raw_training_data)

raw_training_data = raw_training_data.select(cols)

raw_training_data.limit(10).toPandas()

In [ ]:
cols

In [ ]:
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCols=feature_columns,
                        outputCols=[col+"_ohe" for col in feature_columns])

model = encoder.fit(raw_training_data)
encoded = model.transform(raw_training_data)

encoded.limit(10).toPandas()

In [ ]:
cols = encoded.columns

for col_to_remove in feature_columns:
  cols.remove(col_to_remove)

cols

In [ ]:
# cols=raw_training_data.columns
cols.remove("id")
cols.remove("label")

# Let us import the vector assembler
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=cols, outputCol="features")

# Now let us use the transform method to transform our dataset
encoded = assembler.transform(encoded)
encoded.select("features").toPandas()


In [ ]:
from pyspark.ml.feature import StandardScaler

standardscaler = StandardScaler().setInputCol("features").setOutputCol("Scaled_features")
encoded = standardscaler.fit(encoded).transform(encoded)

encoded.select("features","Scaled_features").show(5)

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol="label", maxIter=10, regParam= 0.01)

model=lr.fit(encoded)

predict_train=model.transform(encoded)
# predict_test=model.transform(test)
predict_train.select("label", "prediction", "probability").show(10)

In [ ]:
model.transform(encoded).toPandas()

In [ ]:
from pyspark.ml.classification import LogisticRegression

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import MinMaxScaler

# Create the logistic regression model
lr = LogisticRegression(maxIter=10, regParam= 0.01)

# We create a one hot encoder.
# ohe = OneHotEncoder(
#   # inputCols = ["site_id", "site_domain", "site_category", "app_id", "app_domain", "app_category", "device_id", "device_ip", "device_model"], 
#   inputCols = ["site_domain", "site_category", "app_id", "app_domain", "app_category", "device_id", "device_ip", "device_model"], 
#   # outputCols=["site_id_ohe", "site_domain_ohe", "site_category_ohe", "app_id_ohe", "app_domain_ohe", "app_category_ohe", "device_id_ohe", "device_ip_ohe", "device_model_ohe"]
#   outputCols=["site_domain_ohe", "site_category_ohe", "app_id_ohe", "app_domain_ohe", "app_category_ohe", "device_id_ohe", "device_ip_ohe", "device_model_ohe"]
# )

# Input list for scaling
inputs = ["hour", "C1", "banner_pos", "device_type", "device_conn_type", "C14", "C15", "C16", "C17", "C18", "C19", "C20", "C21"]

# We scale our inputs
assembler1 = VectorAssembler(inputCols=inputs, outputCol="features_scaled1")
scaler = MinMaxScaler(inputCol="features_scaled1", outputCol="features_scaled")

# We create a second assembler for the encoded columns.
assembler2 = VectorAssembler(
  # inputCols=["site_id_ohe", "site_domain_ohe", "site_category_ohe", "app_id_ohe", "app_domain_ohe", "app_category_ohe", "device_id_ohe", "device_ip_ohe", "device_model_ohe", 'features_scaled'], outputCol="features"
  inputCols=["site_domain", "site_category", "app_id", "app_domain", "app_category", "device_id", "device_ip", "device_model", 'features_scaled'], outputCol="features"
)

# Create stages list
# myStages = [assembler1, scaler, ohe, assembler2,lr]
myStages = [assembler1, scaler, assembler2, lr]

# Set up the pipeline
pipeline = Pipeline(stages= myStages)

# We fit the model using the training data.
pModel = pipeline.fit(raw_training_data)

# We transform the data.
trainingPred = pModel.transform(raw_training_data)

# # We select the actual label, probability and predictions
trainingPred.select('label', 'probability', 'prediction').show()

In [ ]:
cols = raw_training_data.columns
cols.remove("label")

# Let us import the vector assembler
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=cols, outputCol="features")

# Now let us use the transform method to transform our dataset
raw_training_data = assembler.transform(raw_training_data)
raw_training_data.select("features").show(truncate=False)

In [ ]:
training_data = raw_training_data.select("features", "label") #.withColumnRenamed("click", "label")
training_data.show()

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [ ]:
model = lr.fit(training_data)

In [ ]:
test = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('../dataset/click-through-rate-prediction/test.gz')

test.printSchema()

In [ ]:
from pyspark.sql.types import StructType, StructField, FloatType, StringType, LongType, IntegerType

schema = StructType([
    StructField('id', LongType(), True),
    StructField("hour", IntegerType(), True),
    StructField("C1", IntegerType(), True),
    StructField("banner_pos", IntegerType(), True),
    StructField("site_id", StringType(), True),
    StructField("site_domain", StringType(), True),
    StructField("site_category", StringType(), True),
    StructField("app_id", StringType(), True),
    StructField("app_domain", StringType(), True),
    StructField("app_category", StringType(), True),
    StructField("device_id", StringType(), True),
    StructField("device_ip", StringType(), True),
    StructField("device_model", StringType(), True),
    StructField("device_type", IntegerType(), True),
    StructField("device_conn_type", IntegerType(), True),
    StructField("C14", IntegerType(), True),
    StructField("C15", IntegerType(), True),
    StructField("C16", IntegerType(), True),
    StructField("C17", IntegerType(), True),
    StructField("C18", IntegerType(), True),
    StructField("C19", IntegerType(), True),
    StructField("C20", IntegerType(), True),
    StructField("C21", IntegerType(), True)
])

# Load test data
test = spark.read.format("csv") \
    .option("header", "true") \
    .schema(schema) \
    .load('../dataset/click-through-rate-prediction/test.gz')

test = test.withColumnRenamed("click", "label")


In [ ]:
pred = pModel.transform(test)

In [ ]:
pred

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(pred)

print("Train Accuracy = %g " % (accuracy))


In [ ]:
from pyspark.ml.feature import StringIndexer

cols = []
pipeline_stages = []
feature_columns = []

for name, type in test.dtypes:
    if type == "string":
        feature_columns.append(f"{name}Index")
        pipeline_stages.append(StringIndexer(inputCol=name, outputCol=f"{name}Index"))
    
        cols.append(f"{name}Index")
    
    else:
        cols.append(name)

cols

In [ ]:
cols

In [ ]:
from pyspark.ml import Pipeline

test = Pipeline(stages=pipeline_stages).fit(test).transform(test)

test = test.select(cols)

test.show()

In [ ]:
cols = test.columns
# cols.remove("click")

# Let us import the vector assembler
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=cols, outputCol="features")

# Now let us use the transform method to transform our dataset
test = assembler.transform(test)
test = test.select("features")

test.show(truncate=False)

In [ ]:
pred = model.transform(test)

In [ ]:
from pyspark.ml.feature import StandardScaler

standardscaler = StandardScaler().setInputCol("features").setOutputCol("Scaled_features")
raw_training_data = standardscaler.fit(raw_training_data).transform(raw_training_data)
raw_training_data.select("features","Scaled_features").show(5)

In [ ]:
from pyspark.ml.feature import HashingTF, Tokenizer, VectorAssembler

# Creating Vector Assembler
vecAssembler = VectorAssembler(inputCols=[col if type != "string" else col+"Index" for col, type in training.dtypes if col != "id"], outputCol="features")
pipeline_stages.append(vecAssembler)


In [ ]:
from pyspark.ml.classification import LogisticRegression

# Creating Logistic Regression Model
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
pipeline_stages.append(lr)


In [ ]:
from pyspark.ml import Pipeline

# Creating pipeline
pipeline = Pipeline(stages=pipeline_stages)


In [ ]:
# Executing pipeline (VectorAssembler + LogisticRegression)
lrModel = pipeline.fit(training)


In [ ]:
stop

In [ ]:

# Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)

pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# Fit the pipeline to training documents.
model = pipeline.fit(training)

# Prepare test documents, which are unlabeled (id, text) tuples.
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

# Make predictions on test documents and print columns of interest.
prediction = model.transform(test)
selected = prediction.select("id", "text", "probability", "prediction")
for row in selected.collect():
    rid, text, prob, prediction = row  # type: ignore
    print(
        "(%d, %s) --> prob=%s, prediction=%f" % (
            rid, text, str(prob), prediction   # type: ignore
        )
    )


In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(training)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

# We can also use the multinomial family for binary classification
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")

# Fit the model
mlrModel = mlr.fit(training)

# Print the coefficients and intercepts for logistic regression with multinomial family
print("Multinomial coefficients: " + str(mlrModel.coefficientMatrix))
print("Multinomial intercepts: " + str(mlrModel.interceptVector))
